In [1]:
# Ken Jee's Playlist Data Science Project From Scratch helped me so much during the cleaning process
# https://www.youtube.com/playlist?list=PL2zq7klxX5ASFejJj80ob9ZAnBHdz5O1t

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from googletrans import Translator
import seaborn as sns

In [2]:
df = pd.read_csv(r"C:\Users\Asaad_Salem\Projects\IMDb-movies\IMDb-movies-data0.csv")

In [3]:
df.columns

Index(['Movie_title', 'Year', 'Run_time', 'Genre', 'Rank', 'Metascore',
       'Certificate', 'Director_Star', 'Gross_revenue'],
      dtype='object')

In [4]:
# Fist of all I'll clean the columns one by one
# 1\ Movie_title: As I can see there are some words that came in Arabic we need to
#    translate that into English

translator = Translator()
Title = df['Movie_title'].apply(lambda x: translator.translate(x, 'en').text)

# After inspecting the column I found one movie title that is mispelled,
# so I have to clean it manually
Title = Title.replace('Did Boits Société', 'Dead Poets Society')
df['Movie_title'] = Title

In [5]:
# 2\ Year: We need to get rid of the parenthesis and change the data type to date

df['Year'] = df['Year'].apply(lambda x: x.replace('(I)', '').strip())
df['Year'] = df['Year'].apply(lambda x: x.replace('(II)', '').strip())
df['Year'] = df['Year'].apply(lambda x: x.replace('(III)', ''))
df['Year'] = df['Year'].apply(lambda x: x.replace('(','').replace(')','').strip())
# We need to drop row 767 since it's a Tv series not a movie
df.drop(767, inplace = True)

df['Year'] = df['Year'].apply(lambda x: int(x))

In [6]:
# 3\ Run_time: we need to get rid of the min to be able to use it for analysis
df['Run_time'] = df['Run_time'].apply(lambda x: int(x.replace('min', '').strip()))

In [7]:
# 4\ Metascore: we need to strip the values as they there is a space. 
df['Metascore'] = df['Metascore'].apply(lambda x: int(x.strip()) if x != '-' else None)

In [8]:
# 5\ Director-Star: we need to split the content of the column
Star = df['Director_Star'].apply(lambda x: x.split('Stars:')[1])
df['Stars'] = Star.apply(lambda x: x.split('-'))

In [9]:
Dir = df['Director_Star'].apply(lambda x: x.split('Stars:')[0])
Dir = Dir.apply(lambda x: x.replace('Director:','').replace('Directors:','').strip())
Dir = Dir.apply(lambda x: x.split('-'))
df['Directors'] = Dir

In [10]:
# 6\ Gross revenue: We need to get rid of the $ sign and the M for million
df['Gross_revenue'] = df['Gross_revenue'].apply(lambda x: x[1:-1])
df['Gross_revenue'] = df['Gross_revenue'].apply(lambda x: float(x) if x else None)

In [11]:
# 7\ Genre: Genre column should be seperated into 3 columns 
df['Genre'] = df['Genre'].apply(lambda x: x.replace('Sci-Fi', 'Sci_Fi'))
df['Genres'] = df['Genre'].str.split('-')

In [12]:
# 8\ Certificate: We need to replace the '-' value by 'Not Rated'
df['Certificate'] = df['Certificate'].apply(lambda x: x.replace('-', 'Not Rated'))

In [13]:
df.columns

Index(['Movie_title', 'Year', 'Run_time', 'Genre', 'Rank', 'Metascore',
       'Certificate', 'Director_Star', 'Gross_revenue', 'Stars', 'Directors',
       'Genres'],
      dtype='object')

In [15]:
# We need to drop unneeded columns
df.drop(['Genre', 'Director_Star'], axis=1, inplace = True)

In [16]:
df.columns

Index(['Movie_title', 'Year', 'Run_time', 'Rank', 'Metascore', 'Certificate',
       'Gross_revenue', 'Stars', 'Directors', 'Genres'],
      dtype='object')

In [17]:
# Number of missing value in each remaining column
df.isnull().sum()

Movie_title       0
Year              0
Run_time          0
Rank              0
Metascore        38
Certificate       0
Gross_revenue    61
Stars             0
Directors         0
Genres            0
dtype: int64

In [18]:
# We can view the rows with missing Metascore values as follows
print(df[df.Metascore.isnull()])

                                           Movie_title  Year  Run_time  Rank  \
65                                    Sunset Boulevard  1950       110   8.4   
67                         Witness for the Prosecution  1957       116   8.4   
83                                  Go and have a look  1985       142   8.3   
98   A death row inmate has escaped or the wind blo...  1956       101   8.3   
163                                               hate  1995        98   8.1   
168                                         White Heat  1949       114   8.1   
174                                      Mary and Max.  2009        92   8.1   
176                             the four hundred Blows  1959        99   8.1   
179       Hearts of Darkness: A Filmmaker's Apocalypse  1991        96   8.1   
231                           Kind Hearts and Coronets  1949       106   8.0   
232                         A Matter of Life and Death  1946       104   8.0   
237                                   Th

In [19]:
# We can view the rows with missing Metascore values as follows
print(df[df.Gross_revenue.isnull()])

               Movie_title  Year  Run_time  Rank  Metascore    Certificate  \
29   It's a Wonderful Life  1946       130   8.6       89.0              G   
65        Sunset Boulevard  1950       110   8.4        NaN  PGNot Rated15   
66          Paths of Glory  1957        88   8.4       90.0              G   
83      Go and have a look  1985       142   8.3        NaN      Not Rated   
89   To Kill a Mockingbird  1962       129   8.3       88.0  PGNot Rated13   
..                     ...   ...       ...   ...        ...            ...   
892                    Ned  2010       124   6.9        NaN      Not Rated   
907           Dog Soldiers  2002       105   6.8        NaN              R   
909      Cemetery Junction  2010        95   6.8        NaN            18+   
935           Julia's eyes  2010       118   6.7        NaN      Not Rated   
953         The Loved Ones  2009        84   6.6       73.0              R   

     Gross_revenue                                             

In [20]:
df.shape

(999, 10)

In [21]:
df.to_csv(r"C:\Users\Asaad_Salem\Projects\IMDb-movies\IMDb-movies-data-cleaned01.csv")